<a href="https://colab.research.google.com/github/MustafaBozkurt84/predictive-maintanance-LSTM/blob/master/Arima1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install statsmodels --upgrade

Requirement already up-to-date: statsmodels in /usr/local/lib/python3.7/dist-packages (0.12.2)


In [2]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pickle

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df =pd.read_csv("/content/drive/MyDrive/Datasets/predictive maintance /31_hidrolikmotoru_analiz.csv")

In [ ]:
df.head()

In [ ]:
df.drop(["name","partno","balancerbasinci","spm","xkurt"],axis=1,inplace=True)

In [ ]:
df.columns=['Time', 'vibx', 'vibz', 'temp', 'zacc', 'zfreq', 'crest']

In [ ]:
#df = pd.read_excel("/content/drive/MyDrive/Datasets/predictive maintance /AnalizDataPM.xlsx",engine='openpyxl')
df["Time"]= [str(i).replace("2020-02-02","2/2/2020").replace("2020-01-02","2/1/2020") for i in df["Time"]]


In [ ]:
df['Time'] = pd.to_datetime(df['Time']) 
df['Time']=[str(i).split(":")[0] for i in df["Time"]]

In [ ]:
df['Time'] = pd.to_datetime(df['Time'],format="%Y-%m-%d %H") #%Y-%m-%d %H:%M:%S

In [ ]:
df=df.groupby("Time").mean()
df.reset_index(inplace=True)
df.to_csv("/content/drive/MyDrive/Datasets/predictive maintance /DataAnaliz.csv")
df.head()

In [ ]:
df.shape

In [ ]:
for col in df.columns[1:]:
  df[col]=[round(i,2)for i in df[col]]

In [ ]:
df1=df.copy()

In [ ]:
df1['Time'] = [str(i).split()[0] for i in df1['Time']]
df1['Time'] = pd.to_datetime(df1['Time'],format="%Y-%m-%d")
df1.set_index("Time",inplace=True)



In [ ]:
df1=df1.groupby("Time").mean()
df1.iloc[:,1].plot()

In [ ]:
#df=pd.read_csv("/content/drive/MyDrive/Datasets/predictive maintance /DataAnaliz.csv",parse_date=True)
#df=df.iloc[:,1:]


# ARIMA and Seasonal ARIMA


## Autoregressive Integrated Moving Averages

The general process for ARIMA models is the following:
* Visualize the Time Series Data
* Make the time series data stationary
* Plot the Correlation and AutoCorrelation Charts
* Construct the ARIMA Model or Seasonal ARIMA based on the data
* Use the model to make predictions

Let's go through these steps!

In [ ]:
df_vibx = df.loc[:,["Time","vibx"]]
df_vibz = df.loc[:,["Time","vibz"]]
df_temp = df.loc[:,["Time","temp"]]
df_zacc = df.loc[:,["Time","zacc"]]
df_crest = df.loc[:,["Time","crest"]]
df_zfreq = df.loc[:,["Time","zfreq"]]

In [ ]:
dataframes=[df_vibx,df_vibz,df_temp,df_zacc,df_crest,df_zfreq]

In [ ]:
for data in dataframes:
    data.set_index("Time",inplace=True)
    data.index = pd.DatetimeIndex(data.index)
    data.plot(figsize=(15, 6))
    plt.title(data.columns[0])

In [ ]:
df.describe()

In [ ]:
from statsmodels.tsa.stattools import adfuller

#Ho: It is non stationary
#H1: It is stationary

def adfuller_test(col,colname):
    print("-"*20 + f"{colname}"+ 20*"-")
    result=adfuller(col)
    labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations Used']
    for value,label in zip(result,labels):
        print(label+' : '+str(value) )
    
    if result[1] <= 0.05:
        print("strong evidence against the null hypothesis(Ho), reject the null hypothesis. Data has no unit root and is stationary")
    else:
        print("weak evidence against null hypothesis, time series has a unit root, indicating it is non-stationary ")
    print(20*"*")


In [ ]:
for col in df.columns[1:]:
  adfuller_test(df.loc[:,col],col)

In [ ]:
from pandas.plotting import autocorrelation_plot
for col in df.columns[1:]:
  autocorrelation_plot(df[col])
  plt.title(f"{col}")
  plt.show()

### Final Thoughts on Autocorrelation and Partial Autocorrelation

* Identification of an AR model is often best done with the PACF.
    * For an AR model, the theoretical PACF “shuts off” past the order of the model.  The phrase “shuts off” means that in theory the partial autocorrelations are equal to 0 beyond that point.  Put another way, the number of non-zero partial autocorrelations gives the order of the AR model.  By the “order of the model” we mean the most extreme lag of x that is used as a predictor.
    
    
* Identification of an MA model is often best done with the ACF rather than the PACF.
    * For an MA model, the theoretical PACF does not shut off, but instead tapers toward 0 in some manner.  A clearer pattern for an MA model is in the ACF.  The ACF will have non-zero autocorrelations only at lags involved in the model.
    
    p,d,q
    p AR model lags
    d differencing
    q MA lags

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
def Autocorrelation_plot(df,col):
    fig = plt.figure(figsize=(12,8))
    ax1 = fig.add_subplot(211)
    fig = plot_acf(df[col],lags=40,ax=ax1)
    ax2 = fig.add_subplot(212)
    ax1.set_title(f" Autocorrelation ({col}) ")
    fig = plot_pacf(df[col],lags=40,ax=ax2)
    ax2.set_title(f" Partial Autocorrelation ({col})")
    

In [ ]:
for col in df.columns[1:]:
  Autocorrelation_plot(df,col)

In [ ]:
!pip install pmdarima

In [ ]:

from pmdarima import auto_arima


In [ ]:

for i in df1.columns:
    print(50*"*")
    print(i)
    stepwise_fit = auto_arima(df1[i], start_p=1, start_q=1,
                           max_p=3, max_q=3, m=7,
                           start_P=0, seasonal=True,
                           d=0, D=0, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)

In [ ]:
 def pickle_all(key,value):
         pickle_out = open("/content/drive/MyDrive/model_predictive_maintanence/"+key+"_arima.pkl", "wb")
         pickle.dump(value, pickle_out)
         pickle_out.close()

In [ ]:
# For non-seasonal data
#p=1, d=1, q=0 or 1
from statsmodels.tsa.arima_model import ARIMA
import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
import statsmodels.api as sm

In [ ]:
def arima_model(df,col,date,date1,p,d,q,p1,d1,q1,m):
    #df.index = pd.DatetimeIndex(df.index)
    df["arıza"]=df[col]
    df["arıza"]=np.where((df["Time"]>"2020-01-27 14:00:00")&(df["Time"] <"2020-01-27 16:00:00"),df[col].max(),df[col].min())
    #model=ARIMA(df[df.index < date][col],order=(p,d,q))
    model=sm.tsa.statespace.SARIMAX(df[(df["Time"]>date)&(df["Time"]<date1)][[col]],order=(p, d, q),seasonal_order=(p1,d1,q1,m))
    
    model_fit=model.fit()
    pickle_all(col,model_fit)
    
    print(model_fit.summary())
    print(df.head())
    df2=df[df.index<1200][[col]]
    df2['forecast_'+col]=model_fit.predict(start=df2.index[0],end=df2.index[-1])
    df2["arıza"]=df["arıza"]
    df2[[col,'forecast_'+col,"arıza"]].iloc[:1200].plot(figsize=(12,8))
    df2["Dif"] = ((df2[col]-df2['forecast_'+col])/df2[col])
    return df2

In [ ]:
df2=arima_model(df,"vibx","2020-02-02 17:00:00","2020-02-21 14:00:00",2,0,2,1,0,1,7*24) #(2,0,2)(1,0,1)

In [ ]:
df2

In [ ]:
arima_model(df,"vibz","2020-01-28 00:00:00","2020-02-21 14:00:00",1,0,0,1,0,1,24*7) #(1,0,0)(1,0,1) 

In [ ]:
arima_model(df,"temp","2020-01-28 00:00:00","2020-02-21 14:00:00",1,0,0,1,0,0,24*7) #(3,0,0)(2,0,0)

In [ ]:
arima_model(df,"zacc","2020-01-28 00:00:00","2020-02-21 14:00:00",1,0,0,1,0,0,24*7) #(3,0,0)(2,0,0)

In [ ]:
arima_model(df,"crest","2020-01-28 00:00:00","2020-02-21 14:00:00",1,0,0,1,0,0,7*24) #(1,0,0)(0,0,0)

In [ ]:
arima_model(df,"zfreq","2020-01-28 00:00:00","2020-02-21 14:00:00",1,0,1,1,0,1,7*24) #(3,0,0)(1,0,0)

In [ ]:
for i in df.columns:
  print(f"{i} sensoru  {df[df[i]==df[i].max()].index[0]} de en yuksek degerı olan {df[i].max()} degerını kaydetmis ")


In [ ]:
"""vibx sensoru  2020-01-27 04:00:00 de en yuksek degerı olan 2.3257780256553273 degerını kaydetmis 
vibz sensoru  2020-01-27 05:00:00 de en yuksek degerı olan 3.027547281323874 degerını kaydetmis 
temp sensoru  2020-01-27 05:00:00 de en yuksek degerı olan 35.27760047281316 degerını kaydetmis 
zacc sensoru  2020-01-27 03:00:00 de en yuksek degerı olan 0.7172621035058485 degerını kaydetmis 
crest sensoru  2020-01-19 13:00:00 de en yuksek degerı olan 10.865886603668711 degerını kaydetmis 
zfreq sensoru  2020-01-27 03:00:00 de en yuksek degerı olan 0.17328324986088234 degerını kaydetmis """